In [14]:
#import os
from dotenv import load_dotenv
# import getpass
# from tqdm.auto import tqdm
load_dotenv()
# import yaml

# config = yaml.safe_load(open("myconfig.yml"))

# os.environ["OPENAI_API_KEY"] = config["OPEN_AI_API_KEY"]
# os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
# os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
# os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
# os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
# os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
# os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
# os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
# os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement langchain_hub (from versions: none)
ERROR: No matching distribution found for langchain_hub
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


True

# Memory

![./images/Screenshot 2024-02-21 at 19.22.19.png](<./images/Screenshot 2024-02-21 at 19.22.19.png>)


- **Sensory Memory:** This component of memory captures immediate sensory inputs, like what we see, hear, or feel. In the context of prompt engineering and AI models, a prompt serves as a transient input, similar to a momentary touch or sensation. It's the initial stimulus that triggers the model's processing.

- **Short-Term Memory:** Short-term memory holds information temporarily, typically related to the ongoing task or conversation. In prompt engineering, this equates to retaining the recent chat history. This memory enables the agent to maintain context and coherence throughout the interaction, ensuring that responses align with the current dialogue.

- **Long-Term Memory:** Long-term memory stores both factual knowledge and procedural instructions. In AI models, this is represented by the data used for training and fine-tuning. Additionally, long-term memory supports the operation of RAG frameworks, allowing agents to access and integrate learned information into their responses. It's like the comprehensive knowledge repository that agents draw upon to generate informed and relevant outputs.

### Adding long-term memory

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings

llm = ChatOllama(
    model = "llama3.2-vision",
    temperature = 0,
    # num_predict = 256,
    # other params ...
)



embeddings = OllamaEmbeddings(model="llama3.2-vision")


In [16]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [17]:
results = retriever.invoke("what are the projects in healthcare?")
results

[Document(id='b3737cb2-9534-4ea7-a0e7-e2f55949c6b8', metadata={'source': 'https://neurons-lab.com/', 'title': 'Neurons Lab | Home', 'description': "We're an AI-exclusive consultancy delivering AI transformation services to guide enterprises into the new era of AI. +100 clients supported.", 'language': 'en-US'}, page_content='Careers    \n\n         Careers    \n\n\n\n         Articles    \n\n         Articles    \n\n\n\n         Contact    \n\n         Contact    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivacy policy\n\n\nCookie notice'),
 Document(id='ed100f7d-868d-4421-96d0-8edf8a7a433e', metadata={'source': 'https://neurons-lab.com/', 'title': 'Neurons Lab | Home', 'description': "We're an AI-exclusive consultancy delivering AI transformation services to guide enterprises into the new era of AI. +100 clients supported.", 'language': 'en-US'}, page_content="Case Studies\n\n    About Us\n\n    Careers\n\n    Articles\n\n\n\n\n\n\n            Let's talk        \n\n\n\n 

### Adding short-term memory

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor

# from langchain.agents import create_openai_functions_agent
#ollama agent insead openai
from langchain.agents import create_react_agent
from langchain.agents import AgentExecutor

from langchain.tools.retriever import create_retriever_tool
from langchain_ollama import ChatOllama
# from langchain_openai import ChatOpenAI
from langchain import hub

In [22]:
# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create a custom prompt for the ReAct agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant. Use the following tools to answer the human's questions: {tools}"),
    ("human", "Hello, I have a question."),
    ("ai", "Hello! I'd be happy to help you with your question. What would you like to know?"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [23]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [31]:
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOllama(
    model = "llama3.2-vision",
    temperature = 0,
    # num_predict = 256,
    # other params ...
)

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create a custom prompt for the ReAct agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant. Use the following tools to answer the human's questions: {tools}. Available tool names: {tool_names}"),
    ("human", "Hello, I have a question."),
    ("ai", "Hello! I'd be happy to help you with your question. What would you like to know?"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt.partial(
        tools=", ".join([tool.name for tool in tools]),
        tool_names=", ".join([tool.name for tool in tools])
    )
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True
)

# Create the message history and the agent with chat history
message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Initialize agent_scratchpad as an empty list
agent_scratchpad = []

# Ensure that tool_names and agent_scratchpad are passed when invoking the agent
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob", "agent_scratchpad": agent_scratchpad},
    config={"configurable": {"session_id": "<foo>"}},
)
agent_with_chat_history.invoke(
    {"input": "what's my name?", "agent_scratchpad": agent_scratchpad},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...


ValueError: variable agent_scratchpad should be a list of base messages, got  of type <class 'str'>

In [29]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

NameError: name 'agent_with_chat_history' is not defined

In [10]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...


wandb: WARNING WARNING: Failed to serialize model: 'int' object is not subscriptable


Your name is Bob! How can I assist you today, Bob?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob! How can I assist you today, Bob?'}